In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
# Parameters
IMG_SIZE = (224, 224)  # Standard size for many pre-trained models
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

In [4]:
# 1. Data Preparation and Augmentation
def create_data_generators(data_dir, batch_size=BATCH_SIZE):
    """
    Create train, validation and test data generators.
    Assumes data_dir has two subdirectories: 'brain_mri' and 'other_scans'
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.15  # 15% for validation
    )

    # Just rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    # Load and split the data
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    valid_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    # Optionally create a separate test set
    # For this example, we'll use the validation set for testing

    return train_generator, valid_generator

In [5]:
# 2. Create Model (Transfer Learning with Fine-tuning)
def create_model():
    """Create a transfer learning model based on ResNet50"""
    # Load pre-trained model without top layers
    base_model = applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )

    # Freeze the base model
    base_model.trainable = False

    # Add custom classification head
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model



In [6]:
# 3. Train model with callbacks
def train_model(model, train_generator, valid_generator, epochs=EPOCHS):
    # Add callbacks for better training
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            'best_brain_mri_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3
        )
    ]

    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=valid_generator,
        callbacks=callbacks
    )

    return history, model



In [7]:
# 4. After initial training, fine-tune the model
def fine_tune_model(model, train_generator, valid_generator, epochs=10):
    # Unfreeze some of the layers in the base model
    base_model = model.layers[0]
    # Unfreeze the last 30 layers
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    # Recompile with a lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE / 10),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    # Continue training
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            'best_fine_tuned_brain_mri_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]

    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=valid_generator,
        callbacks=callbacks
    )

    model.save('mode.h5')  # Save the fine-tuned model

    return history, model


In [8]:
# 5. Evaluation and visualization
def evaluate_model(model, test_generator):
    # Evaluate the model
    results = model.evaluate(test_generator)
    print(f"Test Loss: {results[0]:.4f}")
    print(f"Test Accuracy: {results[1]:.4f}")
    print(f"Test AUC: {results[2]:.4f}")
    print(f"Test Precision: {results[3]:.4f}")
    print(f"Test Recall: {results[4]:.4f}")

    # Get predictions for confusion matrix
    test_generator.reset()
    y_pred = model.predict(test_generator)
    y_pred_classes = (y_pred > 0.5).astype(int)

    return y_pred_classes


In [9]:

# 6. Visualize training history
def plot_training_history(history):
    # Plot accuracy and loss curves
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Accuracy plot
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='lower right')

    # Loss plot
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()


In [ ]:
# Main execution function
def main(data_dir):
    # Create data generators
    train_generator, valid_generator = create_data_generators(data_dir)

    # Create and train initial model
    model = create_model()
    print("Model created. Starting initial training...")

    history, model = train_model(model, train_generator, valid_generator)
    print("Initial training completed. Starting fine-tuning...")

    # Fine-tune model
    fine_tune_history, model = fine_tune_model(model, train_generator, valid_generator)
    print("Fine-tuning completed. Evaluating model...")

    # Evaluate model
    y_pred_classes = evaluate_model(model, valid_generator)

    # Plot training history
    plot_training_history(fine_tune_history)

    # Save the final model
    model.save('final_brain_mri_classifier_model.h5')
    print("Model saved as 'final_brain_mri_classifier_model.h5'")

    return model

# Example usage (commented out - replace with your data directory)
if __name__ == "__main__":
    data_dir = "../../data/mri/"  # Should have subfolders "brain_mri" and "other_scans"
    main(data_dir)